# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096972


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:18,  2.79s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:51,  1.97s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:46,  1.85s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:31,  1.33s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  6.57it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  8.09it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  9.40it/s]

Rendering models:  97%|█████████▋| 29/30 [00:09<00:00, 10.55it/s]

not-logged-in-3b0b929f4475fd276561    0.001521
Jnursssmith5263                       0.001101
Jorgearcelarreta                      0.000330
not-logged-in-f45cae824435ec815f31    0.004541
Jonas_Cross                           0.001081
Linda_J._Berkel                       0.000190
jnarayanbvg                           0.000784
not-logged-in-c6917d9b19f41e6c2995    0.053165
Galaxy-one                            0.000823
not-logged-in-a722508d1bb33173b3c2    0.076481
sn346808                              0.025715
isacami2020                           0.001856
ToriPizza                             0.003502
not-logged-in-14d09f70263c1fe15cc3    0.070453
mami2019                              0.260172
puic                                  0.000922
skotecha                              0.000330
t65yang                               0.004731
ControlledChaos                       0.120314
ranyel.a.e.pasimio                    0.001039
tzele                                 0.001263
ElisabethB   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())